In [87]:
# Import
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
from gensim.models import KeyedVectors
import pickle

In [88]:
train_fn = '/home/eszti/projects/dipterv/panlex/data/smith/train/eng_ita.tsv'
test_fn = '/home/eszti/projects/dipterv/panlex/data/smith/test/eng_ita.tsv'

eng_emb = '/mnt/permanent/Language/Multi/FB/wiki.en/wiki.en.vec'
ita_emb = '/mnt/permanent/Language/Multi/FB/wiki.it/wiki.it.vec'

limit = None

tr_en_fn = 'train_en_emb.txt'
tr_it_fn = 'train_it_emb.txt'

te_en_fn = 'test_en_emb.txt'
te_it_fn = 'test_it_emb.txt'

In [89]:
def read_emb(emb_fn, limit):
    model = KeyedVectors.load_word2vec_format(emb_fn, binary=False, limit=limit)
    return model

In [90]:
def read_word_pairs_tsv(fn, id1, id2, header=True):
    wl1 = set()
    wl2 = set()
    with open(fn) as f:
        lines = f.readlines()
        data = [(line.split()[id1], line.split()[id2]) for i, line in enumerate(lines) if i > 0 or header == False]
    for (w1, w2) in data:
        wl1.add(w1)
        wl2.add(w2)
    return data, wl1, wl2

In [91]:
# saving format: (vocab: ordered list according to frequency, emb_dict: word - nparray dictionary)
def save_filtered_embedding(emb, wl, fn):
    to_save = []
    for w in wl:
        if w in emb:
            to_save.append([w, emb[w], emb.index2word.index(w)])
        else:
            print('not found: {}'.format(w))
    to_save.sort(key=lambda x: x[2])
    dim_x = len(to_save)
    dim_y = 300
    emb_dict = dict()
    vocab = [l[0] for l in to_save]
    for i, l in enumerate(to_save):
        emb_dict[l[0]] = l[1]
    with open(fn, 'wb') as f:
        pickle.dump(file=f, obj=(vocab, emb_dict))

In [92]:
train_wp, tr_en, tr_it = read_word_pairs_tsv(train_fn, 0, 1)
test_wp, te_en, te_it = read_word_pairs_tsv(test_fn, 0, 1)

m_en = read_emb(eng_emb, limit)
m_it = read_emb(ita_emb, limit)

In [93]:
# Train en
print('Saving training en')
save_filtered_embedding(m_en, tr_en, tr_en_fn)
# Train it
print('Saving training it')
save_filtered_embedding(m_it, tr_it, tr_it_fn)
# Test en
print('Saving test en')
save_filtered_embedding(m_en, te_en, te_en_fn)
# Test it
print('Saving test it')
save_filtered_embedding(m_it, te_it, te_it_fn)

Saving training en
Saving training it
not found: prelaurea
Saving test en
Saving test it
not found: oligopolistica
not found: ridimensioni
not found: kostunica


In [94]:
def load(fn):
    with open(fn, 'rb') as f:
        data = pickle.load(f)
        vocab = data[0]
        emb_dict = data[1]
    return vocab, emb_dict

In [98]:
def test_load(fn, message):
    print(message)
    vocab, emb_dict = load(fn)
    print('vocab length: {}'.format(len(vocab)))

In [99]:
test_load(tr_en_fn, 'train en')
test_load(tr_it_fn, 'train it')
test_load(te_en_fn, 'test en')
test_load(te_it_fn, 'test it')

train en
vocab length: 3441
train it
vocab length: 4547
test en
vocab length: 1499
test it
vocab length: 1845
